## Dependecies

In [ ]:
!pip install -q transformers langchain einops huggingface_hub datasets evaluate accelerate  -U

# Login with your huggingface key
import huggingface_hub
huggingface_hub.login(HF_KEY_1) 
huggingface_hub.login(HF_KEY_2)

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Finetuning

In [ ]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    T5ForConditionalGeneration
)

import torch

MODEL_NAME = "Yellow-AI-NLP/komodo-7b-base"
max_input_length = 4096
max_target_length = 512

MODEL_NAME = "aisingapore/sea-lion-7b-instruct"
max_input_length = 2048
max_target_length = 512

MODEL_NAME = "Wikidepia/IndoT5-large"
MODEL_NAME = "Wikidepia/IndoT5-base"
MODEL_NAME = "Wikidepia/IndoT5-small"
MODEL_NAME = "Wikidepia/IndoT5-base-paraphrase"

MODEL_NAME = "Audino/my-awesome-modelv4"
MODEL_NAME = "Audino/my-awesome-modelv4-large"
MODEL_NAME = "Audino/my-awesome-modelv4-small"
MODEL_NAME = "Audino/my-awesome-modelv4-bpara"

MODEL_NAME = "Audino/my-awesomev2-modelv1-base"
max_input_length = 768
max_target_length = 512

MODEL_NAME = "Audino/my-awesomev3-modelv2-10epochs-small"
MODEL_NAME = "Audino/my-awesomev3-modelv2-15epochs-base"
MODEL_NAME = "Audino/my-awesomev3-modelv2-25epochs-bpara"
max_input_length = 864
max_target_length = 64

# MODEL_NAME = "Audino/my-awesomev3-modelv2-10epochs-large"
max_input_length = 640
max_target_length = 64

# TRAINED_MODEL_NAME = "my-awesomev3-modelv2-15epochs-large"

DATASET_ID = 1
# DATASET_ID = 2
DATASET_ID = 3


if (torch.cuda.is_available()):
  device='cuda'
else:
  device='cpu'

In [ ]:
dataset = load_dataset("Audino/news_quotation", use_auth_token=True)

if DATASET_ID == 1:
  split = {
      'train':'train_truncated',
      'test':'test_correction'
  }
elif DATASET_ID == 2:
  split = {
      'train':'train_v2',
      'test':'test_v2'
  }
elif DATASET_ID == 3:
  split = {
      'train':'train_v3',
      'test':'test_v3'
  }

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    # device_map="auto",s
    trust_remote_code=True,
    legacy=False,
)
tokenizer.pad_token = tokenizer.eos_token

model = T5ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
).to(device)

if "Wikidepia" in MODEL_NAME:
  tokenizer.add_tokens(["[", "]", "'"])
  model.resize_token_embeddings(len(tokenizer))

In [ ]:
%%time
def test_model(text=None, label=None):
  if (not text):
    text = str(dataset[split['test']][0]['text'])
    label = str(dataset[split['test']][0]['label'])

  encoded_input = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True, padding=True).to(device)
  output = model.generate(input_ids=encoded_input['input_ids'], max_new_tokens=512)
  print("text:\n", text)
  print("="*40)
  print("answer:\n", label)
  print("="*40)
  print("output:\n", tokenizer.decode(output[-1]))

test_model()

text:
 Jakarta, Beritasatu.com – Peneliti Utama Indikator Politik Indonesia (IPI) Burhanuddin Muhtadi mengatakan Ridwan Kamil dinilai pantas menjadi bakal calon wakil presiden (cawapres) pendamping Ganjar Pranowo berdasarkan hasil survei IPI. RK unggul dengan perolehan angka 16,9 persen, kemudian Erick Thohir menyusul dengan perolehan 16,3 persen dan Sandiaga Uno dengan angka 11,2 persen.  "Kalau kita lihat dua nama teratas adalah Ridwan dan Erick dianggap paling pantas menjadi pendamping Ganjar Pranowo," ujar Burhanuddin Muhtadi saat paparan hasil survei "Swing Voters, Efek Sosialisasi dan Tren Elektoral Jelang Pilpers 2024" secara daring, Sabtu (30/9/2023).  Alasan masyarakat menilai kepantasan ketiga nama teratas bacawapres pendamping Ganjar Pranowo ini diantaranya memiliki kinerja yang baik, cocok dengan sosok Ganjar, baik (ramah, sopan, sederhana), tegas (berwibawa, bijaksana), suka karakter, dan memiliki perhatian kepada rakyat.  Untuk Ridwan Kamil, pada alasan kinerja yang baik 

In [ ]:
def preprocess_function(examples):
    examples['text'] = [str(text) for text in examples['text']]
    examples['label'] = [str(text) for text in examples['label']]

    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        padding='max_length',
        truncation=True,
        # padding=True,
    )
    labels = tokenizer(
        examples["label"],
        max_length=max_target_length,
        padding='max_length',
        truncation=True,
        # padding=True,
    )


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def is_short_text(example):
    return len(tokenizer(example['text'])['input_ids']) <= max_input_length

train_dataset = dataset[split['train']].filter(is_short_text)

train_dataset = train_dataset.map(preprocess_function, batched=True).remove_columns(
    dataset[split['train']].column_names
).shuffle(seed=42)

eval_dataset = dataset[split['test']].map(preprocess_function, batched=True).remove_columns(
    dataset[split['test']].column_names
).shuffle(seed=42)

In [ ]:
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 1
EPOCHS = 5
NUM_OF_LOGS = 10 # how many times showing the logs
LOGGING_STEPS = int(len(train_dataset) * EPOCHS / (NUM_OF_LOGS * BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS))
OUTPUT_DIR = "finetuned_model"

# Show the training loss with every epoch
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    evaluation_strategy="steps",
    save_strategy="steps",
    num_train_epochs=EPOCHS,
    logging_steps=LOGGING_STEPS,
    eval_steps=LOGGING_STEPS,
    save_steps=LOGGING_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="loss",

    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,

    learning_rate=1e-5,
    weight_decay=0.01,
    save_total_limit=1,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    # predict_with_generate=True,
    # generation_max_length=256,
)

trainer = Trainer(
    model=model,
    # tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
1634,0.096000,1.389008
3268,0.115300,1.300886
4902,0.140900,1.232926
6536,0.126900,1.290384


In [ ]:
model.push_to_hub(f"Audino/{TRAINED_MODEL_NAME}")
tokenizer.push_to_hub(f"Audino/{TRAINED_MODEL_NAME}")

In [ ]:
%%time
text = """
'Mahalnya Aura Titiek Soeharto Bersanding dengan Iriana Jokowi, Gelang Giok Jadi Omongan","'Suara.com - Kabar terkini datang dari Titiek Soeharto. Mantan istri Prabowo Subianto ini baru saja membagikan sebuah unggahan yang menarik di Instagram.  Dipantau pada Senin (4/3/2/2024), Titiek Soeharto memamerkan potret dirinya yang menghadiri acara pernikahan dari putra Menteri Pertanian, Amran Sulaiman. Proses pernikahan tersebut sempat viral karena digelar begitu mewah.  Tidak sendiri, Titiek Soeharto ternyata bertemu dengan ibu negara Iriana Jokowi. Foto berdua dari dua perempuan penting di bidang politik ini langsung menjadi perbincangan.  Melalui unggahannya, Titiek Soeharto membongkar impresinya saat berjumpa dengan Iriana Jokowi. Putri dari presiden Soeharto ini mengaku bahagia saat bertemu dengan ibu Gibran Rakabuming Raka tersebut.  ""Senang sekali ketemu dengan Ibu Iriana Ibu negara yang ramah, di acara pernikahan putra menteri Pertanian Bapak Amran Sulaiman,"" tulis Titiek Soeharto.  Titiek Soeharto juga menyematkan sebuah doa untuk dirinya dan Iriana Jokowi. Titiek berharap keduanya bisa selalu sehat.  ""Semoga kita senantiasa diparingi sehat ya, Bu,"" tambah Titiek Soeharto.  Pada pertemuan tersebut, gaya busana Titiek Soeharto memang berbeda dengan Iriana Jokowi. Titiek Soeharto lebih memilih mengenakan kebaya biru ditambah selendang bernuansa jingga.  Sementara itu, Iriana Jokowi memilih pakaian bernuansa coklat berhiaskan brokat. Melengkapi penampilannya, Iriana menambahkan aksesoris berupa kalung.  Dibagikannya foto bersama Iriana Jokowi ini membuat unggahan Titiek Soeharto menuai beragam komentar. Bahkan ada yang salah fokus dengan gelang giok hijau yang selalu dikenakannya.  ""Aura mahalnya Bu Iriana dan Bu Titiek,"" kata netizen.  ""Calon ibu negara nih auranya mahal banget,"" tambah yang lain.  ""Salfok gelang giok hijau dari Prabowo tuh,"" komentar seseorang.  ""Gelang giok hijau dari Prabowo yang dipesan melalui sahabat beliau di Beijing,"" tebak netizen lainnya.  Rumor soal gelang giok hijau yang diberikan oleh Prabowo ini memang belum pernah ditanggapi oleh Titiek Soeharto. Namun Titiek Soeharto sudah lama mengenakan gelang tersebut, terutama saat menghadiri beragam acara resmi.
"""
label = '("Senang sekali ketemu dengan Ibu Iriana Ibu negara yang ramah, di acara pernikahan putra menteri Pertanian Bapak Amran Sulaiman," tulis Titiek Soeharto.,Titiek Soeharto,Iriana Jokowi,Positif)'
# test_model(text, label)
test_model()

In [ ]:
from tqdm import tqdm

def model_evaluation(validation_test=False):
  if validation_test:
    df_result = dataset['test_correction_v2'].to_pandas()
  else:
    df_result = dataset[split['test']].to_pandas()

  df_result["IndoT5_result"] = ""
  for index, row in tqdm(df_result.iterrows(), total=df_result.shape[0]):
    text = row['text']
    label = row['label']

    encoded_input = tokenizer(text, return_tensors='pt').to(device)
    output = model.generate(input_ids=encoded_input['input_ids'], max_new_tokens=768)
    output_text = tokenizer.decode(output[-1])

    # print(output_text)

    df_result.at[index, "IndoT5_result"] = output_text

  return df_result

df_result = model_evaluation(validation_test=True)
df_result


In [ ]:
# df_result.to_csv("IndoT5-bparav2-15epochs.csv")
df_result.to_csv("IndoT5-bparav2-testing.csv")
# df_result.to_csv("IndoT5-large-validation-testing-v4.csv")
# from google.colab import files
# files.download('/content/IndoT5-largev2-validation-testing-v4.csv')

In [ ]:
# df_result.to_csv("IndoT5-bparav2-15epochs.csv")
df_result.to_csv("IndoT5-smallv2-validation-testing-v4.csv")
# df_result.to_csv("IndoT5-large-validation-testing-v4.csv")
from google.colab import files
files.download('/content/IndoT5-smallv2-validation-testing-v4.csv')

In [ ]:
for i, row in df_result.iterrows():
    print("BODY:\n", row['text'])
    print("="*40)

    print("label:\n", row['label'])

    print("="*40)

    print("model_output:\n", row['IndoT5_result'])
    print("\n", "+"*120)



BODY:
 Laporan Wartawan Tribunnews.com, Igman Ibrahim  TRIBUNNEWS.COM, JAKARTA - Capres nomor urut 2, Prabowo Subianto meminta pihak yang mau bergabung menjadi menteri pada kabinet yang akan dipimpinnya harus setuju dengan program makan siang gratis bagi anak-anak yang digagasnya.  Ketua Umum Partai Gerindra itu menyampaikan pihak yang tidak setuju dengan program makan siang gratis seharusnya tidak mengajukan diri masuk ke dalam kabinetnya.  "Saya katakan nanti siapa yang mau masuk kabinet siapa yang mau menjadi menteri saya harus setuju anak-anak harus diberi makan siang kalau kau tidak setuju nggak usah gabung di kabinet Prabowo Subianto," kata Prabowo dalam sambutannya di hadapan puluhan ribu pendukungnya di Stadion GBK, Senayan, Jakarta, Sabtu (10/2/2024).  Ia menuturkan bahwa perjuangan paslon nomor urut 2 sudah sedari awal ingin membawa program makan siang gratis untuk anak-anak Indonesia. Dia pun menuding yang tak setuju program itu sebagai kebangetan.  "Yang tidak setuju mungki